# Isolating black box objective functions

Each black-box objective function in `poli` lives inside a `conda` environment. For example:
- the `foldx` stability objective lives inside `poli__protein`, which has `biopython`, `pdb-tools` and other libraries that are required to interface well with proteins and their representations.
- the `dockstring` objective lives inside `poli__dockstring`, which includes babel as part of its installation.

There are two ways of running objective functions: **having the right dependencies**, or **in isolation**.

[TODO: add diagram]

## Running objective functions in isolation

Let's say that your current environment doesn't have the package `dockstring` installed. You can still run the objective function:

In [3]:
# No output: we don't have it installed.
!pip freeze | grep dockstring

In [7]:
from poli import create

problem_info, f, x0, y0, _ = create(
    name="dockstring",
    target_name="drd2"  # The target protein.
)

poli 🧪: creating an isolated black box function.
poli 🧪: starting the isolated objective process.


/Users/sjt972/anaconda3/envs/poli__dockstring/lib/python3.9/site-packages/dockstring/utils.py:77: DockstringWarning: Although Mac use is supported, docking scores on Mac do not always perfectly match scores from Linux. Therefore, extra care should be taken when comparing results to other platforms. In particular, the baselines in the DOCKSTRING paper were computed on Linux, so please do not directly compare your docking scores to the scores reported on the paper.
  warnings.warn(


As you can see, `poli` warns us that it is creating the conda environment associated with the black box function, and it is creating an isolated process where the objective can be run.

This can be verified by checking the type of `f`:

In [8]:
print(f)

ExternalBlackBox(L=inf, num_evaluations=0)


This black box is an `ExternalBlackBox`, meaning that it is listening to an isolated process running `python` inside the `poli__dockstring` environment.

We can still evaluate `f` like normal:

In [10]:
f(x0)

/Users/sjt972/anaconda3/envs/poli__dockstring/lib/python3.9/site-packages/dockstring/utils.py:77: DockstringWarning: Although Mac use is supported, docking scores on Mac do not always perfectly match scores from Linux. Therefore, extra care should be taken when comparing results to other platforms. In particular, the baselines in the DOCKSTRING paper were computed on Linux, so please do not directly compare your docking scores to the scores reported on the paper.
  warnings.warn(


array([[11.9]])

In [11]:
print(f)

ExternalBlackBox(L=inf, num_evaluations=1)


## When you have the right dependencies...

However, when you have the right dependencies, the `create` method returns the actual objective function. Let's use for example `rdkit_logp`:

In [12]:
problem_info, f_logp, x0_logp, y0_logp, _ = create(
    name="rdkit_logp"
)

poli 🧪: Creating the objective from the repository.


Now, if we print `f_logp`, we can see that the create method returned the actual objective black box.

In [13]:
print(f_logp)

LogPBlackBox(L=inf, num_evaluations=1)


## Conclusions

- `poli` is able to instantiate objective functions inside isolated environments.
- If the dependencies are met, the objective function is created directly instead.

## Read more

You can check the individual dependencies of each black box inside their implementation, or in [their documentation](../objective_repository/all_objectives.md).